# CS 1656 – Introduction to Data Science

## Instructors: Xiaowei Jia, Runlong Yu
## Teaching Assistants: Evangelos Karageorgos, Abdulaziz Alotaibi, Modhumonty Das
### Additional credits: Tahereh Arabghalizi, Zuha Agha, Anatoli Shein, Phuong Pham
## Recitation : SQL via Data API
---
In this recitation, you will execute SQL queries on real data by connecting to the open data portal of [Western Pennsylavnia Regional Data Center](https://www.wprdc.org/) and requesting data via API calls.  

In [ ]:
import json
from datetime import datetime, timedelta, date
import requests
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

We will be using Allegheny County Restaurant/Food Facility Inspection Violation Dataset found here https://data.wprdc.org/dataset/allegheny-county-restaurant-food-facility-inspection-violations. This dataset contains violation data from actual routine inspections by one of health department staff's members for the last two years. It should be fun to find out inspection results for places where we eat in Pittsburgh! =)

In [ ]:
wprdc_api_endpoint = "https://data.wprdc.org/api/3/action/datastore_search_sql"

# id for database table
resource_id = "1a1329e2-418c-4bd3-af2c-cc334e7559af"

# Get the date from 270 days ago)
# end_date = datetime.now()
# start_date = end_date - timedelta(days=270)

# Get two date endpoints
start_date = date(2021, 9, 1)
end_date = date(2022, 6, 1)

# Convert to a string the format the the data center accepts (yyyy-mm-dd)
start_str = start_date.strftime("%Y-%m-%d")
end_str = end_date.strftime("%Y-%m-%d")

# SQL query we'll use in API call to request data
query = """
SELECT *
FROM "{}"
WHERE "inspect_dt" BETWEEN '{}' and '{}' AND "city" = '{}'""".format(resource_id, start_str, end_str, "Pittsburgh")

# Make WPRDC API Call
response = requests.get(wprdc_api_endpoint, {'sql': query}, verify=False)

# Parse response JSON into python dictionary
response_data = json.loads(response.text)

# Convert dictionary to dataframe
df = pd.DataFrame.from_dict(response_data['result']['records'])

# Print the number of rows
print(df.shape[0], "rows total")
print(df.columns)
df.head()


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.wprdc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


10427 rows total
Index(['_id', '_full_text', 'encounter', 'id', 'placard_st', 'facility_name',
       'bus_st_date', 'description', 'description_new', 'num', 'street',
       'city', 'state', 'zip', 'inspect_dt', 'start_time', 'end_time',
       'municipal', 'rating', 'low', 'medium', 'high', 'url'],
      dtype='object')


,_id,_full_text,encounter,id,placard_st,facility_name,bus_st_date,description,description_new,num,...,zip,inspect_dt,start_time,end_time,municipal,rating,low,medium,high,url
0,168171026,'-03':23 '-05':9 '-11':22 '-114':31 '-26':10 '...,202111030038,201706130002,1,Pink Box Bakery Cafe,2017-05-26,Bakery,Cleaning and Sanitization,2104,...,15217,2021-11-03,14:35:00,15:00:00,Pittsburgh-114,V,T,F,F,http://appsrv.alleghenycounty.us/reports/rwser...
1,168171027,'-03':26 '-05':9 '-11':25 '-114':34 '-26':10 '...,202111030038,201706130002,1,Pink Box Bakery Cafe,2017-05-26,Bakery,"Contamination Prevention - Food, Utensils and ...",2104,...,15217,2021-11-03,14:35:00,15:00:00,Pittsburgh-114,V,T,F,F,http://appsrv.alleghenycounty.us/reports/rwser...
2,168171028,'-03':25 '-05':9 '-11':24 '-114':33 '-26':10 '...,202111030038,201706130002,1,Pink Box Bakery Cafe,2017-05-26,Bakery,"Fabrication, Design, Installation and Maintenance",2104,...,15217,2021-11-03,14:35:00,15:00:00,Pittsburgh-114,V,T,F,F,http://appsrv.alleghenycounty.us/reports/rwser...
3,168171029,'-03':22 '-05':9 '-11':21 '-114':30 '-26':10 '...,202111030038,201706130002,1,Pink Box Bakery Cafe,2017-05-26,Bakery,General Premises,2104,...,15217,2021-11-03,14:35:00,15:00:00,Pittsburgh-114,V,T,None,None,http://appsrv.alleghenycounty.us/reports/rwser...
4,168171116,'-01':24 '-06':7 '-101':33 '-13':25 '-15':8 '/...,202201130024,201706160001,1,Yuzu Kitchen,2017-06-15,Restaurant with Liquor,"Fabrication, Design, Installation and Maintenance",409,...,15222,2022-01-13,14:25:00,15:20:00,Pittsburgh-101,V,T,F,F,http://appsrv.alleghenycounty.us/reports/rwser...


Details of useful dataset attributes are below. ((Taken from https://data.wprdc.org/dataset/allegheny-county-restaurant-food-facility-inspection-violations/resource/1a1329e2-418c-4bd3-af2c-cc334e7559af)

* __facility_name__: the name of the facility
* __description__: Facility category
* __description_new__: The name of the potential violation
* __inspect_dt__: Date/time of the inspection
* __rating__: The result of the inspection ('V' for violation, other for non-violation)
* The health risk of a potential violation
 * __low__: low risk
 * __medium__: medium risk
 * __high__: high risk
* The address of the facility
 * __city__: The city
 * __state__: The state
 * __street__: The street
 * __num__: The street number
 * __zip__: The zip code

## Queries

__Q1) Find all unique decription categories of violation in Pittsburgh restaurants over the time span (violation description[violation]).__

In [ ]:
query = """
SELECT DISTINCT "description_new" as violation
FROM "{}"
WHERE "inspect_dt" BETWEEN '{}' and '{}' AND "city" = '{}' """.format(resource_id, start_str, end_str, "Pittsburgh")

response = requests.get(wprdc_api_endpoint, {'sql': query}, verify=False)

df = pd.DataFrame.from_dict(json.loads(response.text)['result']['records'])

df

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.wprdc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,violation
0,Administrative
1,Certified Food Protection Manager
2,Cleaning and Sanitization
3,Cold Holding Temperatures
4,Consumer Advisory
5,"Contamination Prevention - Food, Utensils and ..."
6,Cooking Temperatures
7,Cooling Food
8,Cross-Contamination Prevention
9,Date Marking of Food


__Q2) Find restaurants in Pittsburgh with no violations in at least one decription category (facility name[facility], number of violations[count]). NOTE: a facility has a violation if the inspection rating has the value 'V'.__

In [ ]:
query = """
SELECT "facility_name" as facility, COUNT("description_new") as count
FROM "{}"
WHERE "inspect_dt" BETWEEN '{}' and '{}' AND "city" = '{}' AND "rating" <> '{}'
GROUP BY "facility_name" """.format(resource_id, start_str, end_str, "Pittsburgh", "V")

response = requests.get(wprdc_api_endpoint, {'sql': query}, verify=False)

df = pd.DataFrame.from_dict(json.loads(response.text)['result']['records'])

df

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.wprdc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,facility,count
0,202 Hometown Tacos,1
1,Aladdin's Eatery,1
2,All of Us Care / Volunteers of America,1
3,Bao,1
4,Bar Marco @ the Firehouse,1
...,...,...
69,UPMC Mercy / 1847 Cafe,1
70,Vickey's Soul Grill Restaurant & Catering,1
71,Victory Banquet Hall,1
72,Vocelli Pizza,2


# Tasks

__Tasks 1 to 4 must be implemented in Task.py.__

__For all tasks, we want the results in Pittsburgh, over the specified time span, and all queries are about violations (rating is 'V').__

__As the API returns the results as a list of dictionaries, the order of the query columns is irrelevant.__


__T1) Find the top 20 facilities that start with 'Pitt' and have the highest counts of violations _(facility name[facility], number of violations[count])_.__

In [ ]:
query = """
SELECT "facility_name" as facility, COUNT("description_new") as count
FROM "{}"
WHERE "inspect_dt" BETWEEN '{}' and '{}' AND "city" = '{}' AND "rating" = 'V' AND "facility_name" LIKE '%Pitt%'
GROUP BY "facility_name"
ORDER BY count DESC
LIMIT 20""".format(resource_id, start_str, end_str, "Pittsburgh")

response = requests.get(wprdc_api_endpoint, {'sql': query}, verify=False)

df = pd.DataFrame.from_dict(json.loads(response.text)['result']['records'])

df

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.wprdc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,facility,count
0,Pitt Eats Litchfield Tower (B) / The Eatery @ ...,7
1,Pitt Eats William Pitt Union / Schenley Cafe &...,7
2,Wings Over Pittsburgh Southside,7
3,Catapult Greater Pittsburgh Commercial Commiss...,7
4,Vallozzi's Pittsburgh / Earth Inspired Salads,6
5,Wellness Pavilion @ Pitt Community Engagement ...,6
6,Renaissance Pittsburgh / Braddock's American B...,5
7,Pitt Eats Commissary / Posvar Kitchen,4
8,Pitt Eats Cathedral of Learning / Cathedral Ca...,4
9,Renaissance Pittsburgh / Hotel Main Kitchen,4


__T2) Find the top 18 restaurants with the maximum number of violations _(facility name[facility], number of violations[count])_. Include all results in case of a tie (For example, if the 18th top restaurant has 10 violations, incude all other restaurants with 10 violations). HINT: You will need an extra query to get the tie-breaker value.__

In [ ]:
query = """
SELECT COUNT("description_new")
FROM "{}"
WHERE "inspect_dt" BETWEEN '{}' AND '{}' AND "city" = '{}' AND "rating" = 'V'
GROUP BY "facility_name"
ORDER BY COUNT("description_new") DESC
LIMIT 1 OFFSET 17""".format(resource_id, start_str, end_str, "Pittsburgh")

response1 = requests.get(wprdc_api_endpoint, {'sql': query}, verify=False)
tie_breaker_count = json.loads(response1.text)['result']['records'][0]['count']

query1 = """
SELECT "facility_name" AS facility, COUNT("description_new") AS count
FROM "{}"
WHERE "inspect_dt" BETWEEN '{}' AND '{}' AND "city" = '{}' AND "rating" = 'V'
GROUP BY "facility_name"
HAVING COUNT("description_new") >= {}
ORDER BY count DESC""".format(resource_id, start_str, end_str, "Pittsburgh", tie_breaker_count)

response2 = requests.get(wprdc_api_endpoint, {'sql': query1}, verify=False)

df = pd.DataFrame.from_dict(json.loads(response2.text)['result']['records'])

df

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.wprdc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.wprdc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,facility,count
0,A's 2's / Ace's & Deuce's Lounge,58
1,Caliente Pizza & Draft House,48
2,Subway Sandwiches & Salads,46
3,Shrodi's Cucina di Maria,44
4,Uptown Kitchen,40
5,Jimmy Wan's Restaurant & Lounge,36
6,Ocean Treasures,35
7,Silk Elephant,34
8,California Taco Shop - Shaler,33
9,Piada Italian Street Food,32


__T3) Find the facilities that start with 'Pitt' and have violations over the time span _(violation description[violation], number of facilities[count], facility names[facilities])_. The _facilities_ field must be a concatenation of all facility names, in alphabetical order, seperated by a comma and a space (', ').__

In [ ]:
query = """
SELECT "description_new" as violation, COUNT(DISTINCT "facility_name") as count, STRING_AGG("facility_name", ', ' ORDER BY "facility_name" ASC) as facilities
FROM "{}"
WHERE "inspect_dt" BETWEEN '{}' and '{}' AND "city" = '{}' AND "rating" = 'V'
GROUP BY "description_new"
""".format(resource_id, start_str, end_str, "Pittsburgh")

response = requests.get(wprdc_api_endpoint, {'sql': query}, verify=False)

df = pd.DataFrame.from_dict(json.loads(response.text)['result']['records'])

df

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.wprdc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,violation,count,facilities
0,Administrative,285,"2200 Club, Swissvale Loyal Order Of Moose No. ..."
1,Certified Food Protection Manager,356,"202 Hometown Tacos, 61 C Cafe, 7 Eleven 167, 7..."
2,Cleaning and Sanitization,929,"1:11 Juice Bar, 1:11 Juice Bar, 202 Hometown T..."
3,Cold Holding Temperatures,338,"202 Hometown Tacos, 61 C Cafe, 7 Eleven 167, 7..."
4,Consumer Advisory,32,"All American Bistro, All American Bistro, Benz..."
5,"Contamination Prevention - Food, Utensils and ...",527,"61 C Cafe, 7-Eleven #36122A, 7-Eleven #36160, ..."
6,Cooking Temperatures,2,"Central Diner & Grill, Omar’s Halal Grill"
7,Cooling Food,142,"61 C Cafe, 876 Island Spice, All American Bist..."
8,Cross-Contamination Prevention,179,"Adda Bazaar, Aldi #69, Aldi #79, Alex's Pit St..."
9,Date Marking of Food,303,"7 Eleven #35968A, 876 Island Spice, Alex's Pit..."


Now lets look at all facilities that contain word 'Pitt'.

__T4) Find the category descriptions and their high, medium, low risk ratings for all violations at all facilities that have word 'Pitt' in their name. Note that results that contain word 'Pitt' as part of another word (e.g. 'Pittsburgh') should not be included _(facility name[facility], violation description[violation], high[high], medium[medium], low[low])_. HINT: consider _all_ edge cases for identifying 'Pitt' as a seperate word.__

In [ ]:
query = """
SELECT "facility_name" as facility, "description_new" as violation, "high" as high, "medium" as medium, "low" as low
FROM "{}"
WHERE "inspect_dt" BETWEEN '{}' and '{}' AND "city" = '{}' AND "rating" = 'V' AND ("facility_name" LIKE '%Pitt%' OR "facility_name" LIKE 'Pitt %' OR "facility_name" LIKE '%Pitt') AND ("facility_name" NOT LIKE 'Pittsburgh%' AND "facility_name" NOT LIKE '%Pittsburgh' AND "facility_name" NOT LIKE '%Pittsburgh%')
""".format(resource_id, start_str, end_str, "Pittsburgh")

response = requests.get(wprdc_api_endpoint, {'sql': query}, verify=False)

df = pd.DataFrame.from_dict(json.loads(response.text)['result']['records'])

df

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.wprdc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,facility,violation,high,medium,low
0,Pitt Eats / Litchfield Tower B / Litchfield Co...,Walls and ceilings,None,None,T
1,Pitt Eats Mervis Hall / Bottom Line Bistro,"Fabrication, Design, Installation and Maintenance",F,F,T
2,Pitt Eats Mervis Hall / Bottom Line Bistro,Walls and ceilings,None,None,T
3,Pitt Eats Sutherland Hall / Market @ Sutherland,Plumbing,F,F,T
4,Pitt Eats Sutherland Hall / Market @ Sutherland,"Fabrication, Design, Installation and Maintenance",F,F,T
5,Pitt Hillman Library Ground Level / Saxbys,Handwashing Facilities,F,F,T
6,Pitt Hillman Library Ground Level / Saxbys,"Fabrication, Design, Installation and Maintenance",F,F,T
7,Pitt Eats Benedum Hall / Einstein Bros. Bagels,"Fabrication, Design, Installation and Maintenance",F,F,T
8,Pitt Eats William Pitt Union / Schenley Cafe &...,Hot Holding Temperatures,F,T,F
9,Pitt Eats William Pitt Union / Schenley Cafe &...,"Fabrication, Design, Installation and Maintenance",F,F,T
